In [ ]:
%pylab inline

In [ ]:
# Much of this can be ignored: Was used to match Chloe's notebook, but really the point
# is to just do arcsin of angles between 0 and 20 deg.

num_steps=800 #6400 #6400*2
linsp=np.linspace(0,1,num_steps)
time_interval_usec = 3.0 / num_steps * 1000000


num_steps_per_range = 6400

realpos=linsp * 4.4
theta=linsp*90
theta_prime = theta
tim=linsp*1.5

steps_per_pos = 4.4 / num_steps_per_range

theta_prime[0:3],theta_prime[-3:] #+0.000688
pos=np.sin(radians(theta_prime))
pos2=np.concatenate((pos[:-1],pos[::-1]) )
tim2=np.concatenate((tim,tim[-1]+tim[1:]) )

In [ ]:
np.arcsin(radians(0))

In [ ]:
plt.plot(tim2,pos2,'-')
plt.xlabel('Time',size=14)
plt.ylabel('Pos',size=14)

# 6400=8000
# 3200=4000
# 1600=2000
#  800=1000
pos_divs=np.linspace(1-radians(20),1,800) # 3200

theta_divs=np.arcsin(pos_divs)
time_desired=theta_divs/(np.pi/2.0)*1.5
plt.plot(time_desired, pos_divs, 'o')

plt.grid()
#plt.xlim(0,1.51)

#plt.xlim(0,1.5)
ax2=plt.twiny();
plt.xlim(0,1*20.0)

plt.figure()

delays=diff(time_desired)*1000000

plt.plot(delays, 'o-')
plt.ylabel('Time between steps', size=14)
plt.xlabel('Elapsed Time', size=14)
plt.semilogy()
plt.axhline(2**16+0,color='k')
plt.axhline(0,color='k')
print( '%6.2f'%np.min( delays), '%6.2f'%np.max(delays[:-1]), '%6.2f'%np.sum(delays))
print( delays[:5])
print( delays[-5:])


In [ ]:
intervals_usec =  delays

# Scale the intervals to uint8 for the table, going from 0-254 (linear)
# We lose precision, of course, but hopefully with rounding are
# still close to the desired behavior.

INTERVAL_TABLE_MAX=254.0 # uint8, from 0-254
INTERVAL_TABLE_MAX=2**16 # uint8, from 0-254
INTERVAL_TABLE_MAX=np.max(delays)-np.min(delays)

expander_exponent=0 # To get a more accurate int, multiply/divide by this (2k)

interval_min=np.round( np.min(intervals_usec) )
interval_max=np.max(intervals_usec)
scaler=((interval_max-interval_min)/INTERVAL_TABLE_MAX)
scaler_expanded=np.round( scaler*(2**expander_exponent) )
if scaler==0: scaler=1 # For constants scalar might be 0

intervals_int8=np.round( (intervals_usec-interval_min)/scaler )
intervals_int8_recon = np.array(
    intervals_int8 * scaler_expanded/(2**expander_exponent) + interval_min, dtype=int)

interval_min,interval_max,scaler,scaler_expanded,intervals_int8,np.sum(intervals_int8_recon)

In [ ]:
hist((intervals_int8[0:3000]), bins=range(255) )

In [ ]:
plot(intervals_int8_recon,'.')
plot(delays)
#plt.ylim(0.01,0.015)
plt.semilogy()


In [ ]:
fil=open('lookup_table2.h','wt')
fil.writelines('const unsigned int table2_info[] = {\n')
fil.writelines('\t%d, //num_steps\n'%num_steps);
fil.writelines('\t%d, //const unsigned int table_scaler = ;\n'%scaler_expanded);
fil.writelines('\t%d, //const unsigned int table_expander_exponent = ;\n'%expander_exponent);
fil.writelines('\t%d}; //const unsigned int table_interval_min = };\n'%interval_min);
fil.writelines('const PROGMEM uint16_t table2[] = {\n');
for nelement in intervals_int8:
    fil.writelines('%d,\n'%nelement)
fil.writelines('};\n');
fil.close()

# In reverse:
fil=open('lookup_table2r.h','wt')
fil.writelines('const unsigned int table2r_info[] = {\n')
fil.writelines('\t%d, //num_steps\n'%num_steps);
fil.writelines('\t%d, //const unsigned int table_scaler = ;\n'%scaler_expanded);
fil.writelines('\t%d, //const unsigned int table_expander_exponent = ;\n'%expander_exponent);
fil.writelines('\t%d}; //const unsigned int table_interval_min = };\n'%interval_min);
fil.writelines('const PROGMEM uint16_t table2r[] = {\n');
for nelement in intervals_int8[::-1]:
    fil.writelines('%d,\n'% (nelement) )
fil.writelines('};\n');
fil.close()

In [ ]:
#y=sin(x)
#x=arcsin(y)

In [ ]:
tim2p=tim2[:-1] # Remove last element of time, since diff skips one elmt
delta_pos = np.diff(pos2)
delta_steps = abs(delta_pos)/steps_per_pos

In [ ]:
delta_steps[0:10]

In [ ]:
delta_steps[ delta_steps<=1 ]=np.ceil(delta_steps[ delta_steps<=1 ]*2.0)/2.0

plot(tim2p,delta_steps,'.')
plt.ylabel('Steps to move', size=14)
plt.xlabel('Time', size=14)

print( np.sum(delta_steps))

In [ ]:
delta_pos = np.diff(pos)
delta_steps = abs(delta_pos)/steps_per_pos
plt.plot(tim[1:],delta_steps)

m,b=np.polyfit(tim[1:],delta_steps,1)

plt.plot(tim[1:],tim[1:]*m+b,'r-')

plt.loglog()

In [ ]:
m,b
# y=mx+b
# x=(y-b)/m

In [ ]:
# In each epoch between dots, how many steps do we need to fit in
# that time interval?
time_between_steps = time_interval_usec/(delta_steps)
plot(tim2p,time_between_steps,'.')
plt.ylabel('Time between steps (usec)', size=14)
#plt.xlabel('Time', size=14)
plt.semilogy()

In [ ]:
print( np.min(time_between_steps) )
print( np.max(time_between_steps) )
print( np.sum(time_between_steps))

In [ ]:
print (time_between_steps[0:5])
half=len(time_between_steps)//2
print (time_between_steps[half-5:half])

In [ ]:
np.sum(time_between_steps)